In [1]:
#!pip install rioxarray
import dask.array as da
import pystac_client
from pystac_client import Client
import leafmap
from datetime import datetime
import dask
import planetary_computer as pc
import geogif
import numpy as np
import xarray as xr
import rioxarray
import geopandas as gpd
import matplotlib.pyplot as plt
import geojson
import json
from geogif import dgif, gif
import pandas as pd
import shapely
from shapely.geometry import mapping
from shapely.geometry import box
import folium
from pystac import ItemCollection
from pyproj import CRS
from branca.colormap import LinearColormap


In [2]:
# Read the shapefiles in as GeoDataFrames
countyCensus = gpd.read_file('CorrectWorcester.shp')
boundary = gpd.read_file('Boundary/WorcesterBoundary.shp')

In [3]:
# Check CRSs
print(boundary.crs)
print(countyCensus.crs)

EPSG:3585
EPSG:3857


In [4]:
# We'll probably go with 3857 as the CRS to work in here, as it is preferable for web mapping
# Reproject the boundary layer to EPSG:3857
boundary = boundary.to_crs(epsg=3857)

In [5]:
# Clip countyCensus to the city boundary
worcester2020Census = gpd.clip(countyCensus, boundary)

In [6]:
# # Lets check out worcesterCensus
# print((worcester2020Census.columns).tolist())

In [7]:
# It has a lot of columns, most of which are, for the purposes of this project, unecessary
# Rename the ones we need
# I am referencing https://www.arcgis.com/home/item.html?id=abd94a6cc94645f88811ae91802909a0
# and the column aliases as vieweD in the attribute table

# Mapping of old column names to new column names
column_mapping = {
    'P0010001': 'Total Pop',
    'P0010002': 'One race',
    'P0010003': 'White alone',
    'P0010004': 'Black',
    'P0010005': 'American Indian',
    'P0010006': 'Asian',
    'P0010007': 'Pacific Islander',
    'P0010008': 'Other Race',
    'P0020002': 'Hispanic',
    'PCT_P00300': 'Percent 18 Years and Over',
    'PCT_P00200': 'Percent Hispanic',
    'PCT_P002_1': 'Percent White',
    'PCT_P002_2': 'Percent Black',
    'PCT_P002_3': 'Percent American Indian',
    'PCT_P002_4': 'Percent Asian',
    'PCT_P002_5': 'Percent Pacific Islander',
    'PCT_P002_6': 'Percent Other Race',
    'PCT_P002_7': 'Percent two or more races',
    'PCT_H00100': 'Percent of Housing Occupied',
    'PCT_H001_1': 'Percent of Housing Vacant'
}

# Rename columns
worcester2020Census = worcester2020Census.rename(columns=column_mapping)


In [8]:
# Now lets get rid of unecessary columns
# Filter out columns that start with "P00"
columns_to_keep = [col for col in worcester2020Census.columns if not (col.startswith('P00') or col.startswith('H00'))]

# Select only the columns you want to keep
worcester2020Census = worcester2020Census[columns_to_keep]


In [9]:
print((worcester2020Census.columns).tolist())

['GEOID', 'NAME', 'County_Nam', 'State_Name', 'Total Pop', 'One race', 'White alone', 'Black', 'American Indian', 'Asian', 'Pacific Islander', 'Other Race', 'Hispanic', 'Percent 18 Years and Over', 'Percent Hispanic', 'Percent White', 'Percent Black', 'Percent American Indian', 'Percent Asian', 'Percent Pacific Islander', 'Percent Other Race', 'Percent two or more races', 'Percent of Housing Occupied', 'Percent of Housing Vacant', 'SUMLEV', 'REGION', 'DIVISION', 'STATE', 'STATENS', 'COUNTY', 'COUNTYCC', 'COUNTYNS', 'COUSUB', 'COUSUBCC', 'COUSUBNS', 'SUBMCD', 'SUBMCDCC', 'SUBMCDNS', 'ESTATE', 'ESTATECC', 'ESTATENS', 'CONCIT', 'CONCITCC', 'CONCITNS', 'PLACE', 'PLACECC', 'PLACENS', 'TRACT', 'AIANHH', 'AIHHTLI', 'AIANHHFP', 'AIANHHCC', 'AIANHHNS', 'AITS', 'AITSFP', 'AITSCC', 'AITSNS', 'TTRACT', 'ANRC', 'ANRCCC', 'ANRCNS', 'CBSA', 'MEMI', 'CSA', 'METDIV', 'NECTA', 'NMEMI', 'CNECTA', 'NECTADIV', 'CBSAPCI', 'NECTAPCI', 'UA', 'UATYPE', 'UR', 'CD116', 'SLDU18', 'SLDL18', 'VTD', 'VTDI', 'ZCTA', 

In [10]:
# Get rid of More columns
columns_to_delete = ['County_Nam', 'State_Name','SUMLEV', 'REGION', 'DIVISION', 'STATE', 'STATENS', 'COUNTY', 'COUNTYCC', 'COUNTYNS', 'COUSUB', 'COUSUBCC', 'COUSUBNS', 'SUBMCD', 'SUBMCDCC', 'SUBMCDNS', 'ESTATE', 'ESTATECC', 'ESTATENS', 'CONCIT', 'CONCITCC', 'CONCITNS', 'PLACE', 'PLACECC', 'PLACENS', 'TRACT', 'AIANHH', 'AIHHTLI', 'AIANHHFP', 'AIANHHCC', 'AIANHHNS', 'AITS', 'AITSFP', 'AITSCC', 'AITSNS', 'TTRACT', 'ANRC', 'ANRCCC', 'ANRCNS', 'CBSA', 'MEMI', 'CSA', 'METDIV', 'NECTA', 'NMEMI', 'CNECTA', 'NECTADIV', 'CBSAPCI', 'NECTAPCI', 'UA', 'UATYPE', 'UR', 'CD116', 'SLDU18', 'SLDL18', 'VTD', 'VTDI', 'ZCTA', 'SDELM', 'SDSEC', 'SDUNI', 'PUMA', 'AREALAND', 'AREAWATR', 'AWATER', 'ALAND', 'INTPTLON', 'INTPTLAT', 'BASENAME', 'FUNCSTAT', 'GCUNI', 'POP100', 'HU100', 'PARTFLAG', 'UGA', 'Percent 18 Years and Over', 'Percent two or more races', 'Percent of Housing Occupied', 'Percent of Housing Vacant', 'NAME','Shape__Are', 'Shape__Len',]

worcester2020Census = worcester2020Census.drop(columns=columns_to_delete)

In [11]:
print((worcester2020Census.columns).tolist())

['GEOID', 'Total Pop', 'One race', 'White alone', 'Black', 'American Indian', 'Asian', 'Pacific Islander', 'Other Race', 'Hispanic', 'Percent Hispanic', 'Percent White', 'Percent Black', 'Percent American Indian', 'Percent Asian', 'Percent Pacific Islander', 'Percent Other Race', 'geometry']


In [12]:
# create a nice 'Tract' column
worcester2020Census['GEOID'] = worcester2020Census['GEOID'].astype(str)

# Extract the last 6 digits of each 'GEOID' and assign it to the 'Tract' column
worcester2020Census['Tract'] = worcester2020Census['GEOID'].str[-6:]

In [13]:
# 'Tract' is now the common identifier between 2010 and 2020, so we can get rid of GEOID
worcester2020Census = worcester2020Census.drop(columns='GEOID')

In [14]:
worcester2020Census.sort_values(by='Tract')

,Total Pop,One race,White alone,Black,American Indian,Asian,Pacific Islander,Other Race,Hispanic,Percent Hispanic,Percent White,Percent Black,Percent American Indian,Percent Asian,Percent Pacific Islander,Percent Other Race,geometry,Tract
58,5004,4719,4290,190,11,73,1,154,303,6.1,84.9,3.5,0.2,1.4,0.0,0.8,"MULTIPOLYGON (((-8001131.688 5204000.758, -800...",727100
91,5388,5083,4646,101,17,245,2,72,249,4.6,85.3,1.8,0.1,4.5,0.0,0.2,"POLYGON ((-8001114.990 5204014.601, -8001131.6...",728100
92,4045,3833,3553,80,2,139,0,59,155,3.8,86.6,2.0,0.0,3.3,0.0,0.5,"POLYGON ((-7995293.535 5208823.946, -7995374.3...",728200
129,5008,4802,4353,227,5,117,1,99,380,7.6,82.2,4.2,0.1,2.3,0.0,0.7,"POLYGON ((-7989207.029 5207720.620, -7989287.5...",729100
130,2869,2657,2473,66,4,32,0,82,203,7.1,84.5,1.9,0.0,1.0,0.0,0.7,"POLYGON ((-7991198.981 5212202.166, -7991250.9...",729200
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
88,5381,5055,4744,119,5,130,4,53,227,4.2,86.9,2.0,0.1,2.4,0.1,0.3,"POLYGON ((-7992411.027 5192486.506, -7992417.3...",737100
90,6690,6299,5675,189,9,298,1,127,290,4.3,83.8,2.7,0.0,4.4,0.0,0.7,"POLYGON ((-7990857.785 5193411.299, -7990858.8...",737300
133,5286,4958,3433,54,4,1361,0,106,220,4.2,64.1,0.8,0.0,25.7,0.0,1.0,"MULTIPOLYGON (((-7985084.866 5197193.020, -798...",739101
139,7945,7477,5837,116,14,1395,4,111,300,3.8,73.0,1.4,0.1,17.6,0.0,0.6,"POLYGON ((-7989207.029 5207720.620, -7989200.5...",739500


In [15]:
# hmm, 2010 census only has 60 tracts
worcester2020Census['Tract'].nunique()

61

In [16]:
# Lets visualize to investigate
# To visualize this data we have to reproject to 4326, which is the best CRS for folium
worcester2020Census4326 = worcester2020Census.to_crs(epsg=4326)

In [17]:
import folium

# Create a Folium map
m = folium.Map(location=[42.2626, -71.8023], zoom_start=10)
colormap = LinearColormap(colors=['green', 'white'], vmin=17.7, vmax=88.7)

# Iterate over the rows of the worcesterCensus4326 GeoDataFrame and add polygons to the map with color based on the "Percent White" column
for idx, row in worcester2020Census4326.iterrows():
    # Check the value of "Percent White" for the current row
    print(f"Census Tract: {row['Tract']}, Percent White: {row['Percent White']}")
    
    # Style function for GeoJSON features
    style_function = lambda x, row=row: {
        'fillColor': colormap(row['Percent White']),
        'color': 'black',
        'weight': 1,
        'fillOpacity': 0.7
    }
    # Add GeoJSON feature to the map
    #folium.GeoJson(row.geometry.__geo_interface__, style_function=style_function).add_to(m)
    # Add GeoJSON feature to the map with tooltip showing "Percent White" value
    
    folium.GeoJson(row.geometry.__geo_interface__, 
                   style_function=style_function,
                   tooltip=f"Census Tract: {row['Tract']}<br>Percent White: {row['Percent White']}").add_to(m)
    
#folium.GeoJson(combined_gdf.geometry).add_to(m)
# Display the map
m


Census Tract: 737300, Percent White: 83.8
Census Tract: 732801, Percent White: 61.3
Census Tract: 761300, Percent White: 81.9
Census Tract: 732301, Percent White: 62.5
Census Tract: 737100, Percent White: 86.9
Census Tract: 736400, Percent White: 83.6
Census Tract: 732901, Percent White: 49.4
Census Tract: 732802, Percent White: 49.0
Census Tract: 732902, Percent White: 68.9
Census Tract: 732700, Percent White: 34.1
Census Tract: 733000, Percent White: 26.3
Census Tract: 732600, Percent White: 31.2
Census Tract: 731300, Percent White: 17.7
Census Tract: 732302, Percent White: 47.6
Census Tract: 739101, Percent White: 64.1
Census Tract: 732202, Percent White: 60.0
Census Tract: 732201, Percent White: 55.6
Census Tract: 732002, Percent White: 53.7
Census Tract: 731900, Percent White: 28.8
Census Tract: 730402, Percent White: 38.6
Census Tract: 730500, Percent White: 50.0
Census Tract: 730401, Percent White: 40.3
Census Tract: 730300, Percent White: 64.6
Census Tract: 730200, Percent Whit

In [18]:
# Looks like Tract 731600, right in the middle of Worcester, was split along Highland Street into 731601 and 731602,

# In order to compare 2020 with 2010 we will have to merge these back into one.

# Filter rows for Tract 731601 and Tract 731602
tract_731601 = worcester2020Census4326[worcester2020Census4326['Tract'] == '731601']
tract_731602 = worcester2020Census4326[worcester2020Census4326['Tract'] == '731602']

# Extract geometry
t1 = tract_731601['geometry']
t2 = tract_731602['geometry']
# Concatenate
combined_rows = pd.concat([t1, t2], ignore_index=True)
# Unify
combined = combined_rows.unary_union
combined_gdf = gpd.GeoDataFrame(geometry=[combined])
combined_gdf= combined_gdf.set_crs("EPSG:4326")
print(type(combined_gdf))
combined_gdf

<class 'geopandas.geodataframe.GeoDataFrame'>


,geometry
0,"POLYGON ((-71.81752 42.26785, -71.81746 42.268..."


In [19]:
tract_731601

,Total Pop,One race,White alone,Black,American Indian,Asian,Pacific Islander,Other Race,Hispanic,Percent Hispanic,Percent White,Percent Black,Percent American Indian,Percent Asian,Percent Pacific Islander,Percent Other Race,geometry,Tract
107,4812,4252,2637,516,28,583,2,486,1022,21.2,51.0,10.0,0.3,12.1,0.0,1.1,"POLYGON ((-71.81767 42.26747, -71.81752 42.267...",731601


In [20]:
# Create a dictionary to store the sums of each column
sums = {}

# List of columns to sum
columns_to_sum = ['Total Pop', 'One race', 'White alone', 'Black', 'American Indian', 'Asian', 'Pacific Islander', 'Other Race', 'Hispanic']

# Iterate over each column and calculate the sum
for column in columns_to_sum:
    sums[column] = tract_731601[column].sum() + tract_731602[column].sum()

# Add the sums as new columns to combined_gdf
for column, value in sums.items():
    combined_gdf[column] = value

# Print the updated combined_gdf
combined_gdf


,geometry,Total Pop,One race,White alone,Black,American Indian,Asian,Pacific Islander,Other Race,Hispanic
0,"POLYGON ((-71.81752 42.26785, -71.81746 42.268...",8079,7340,4906,887,31,887,2,627,1376


In [21]:
total_pop = combined_gdf['Total Pop']
combined_gdf['Percent Hispanic'] = ((combined_gdf['Hispanic'] / total_pop) * 100).round(2)
combined_gdf['Percent White'] = ((combined_gdf['White alone'] / total_pop) * 100).round(2)
combined_gdf['Percent Black'] = ((combined_gdf['Black'] / total_pop) * 100).round(2)
combined_gdf['Percent American Indian'] = ((combined_gdf['American Indian'] / total_pop) * 100).round(2)
combined_gdf['Percent Asian'] = ((combined_gdf['Asian'] / total_pop) * 100).round(2)
combined_gdf['Percent Pacific Islander'] = ((combined_gdf['Pacific Islander'] / total_pop) * 100).round(2)
combined_gdf['Percent Other Race'] = ((combined_gdf['Other Race'] / total_pop) * 100).round(2)
combined_gdf['Tract'] = '731600'


In [22]:
type(combined_gdf)

geopandas.geodataframe.GeoDataFrame

In [23]:
# Ok, looks like it's good to go
combined_gdf

,geometry,Total Pop,One race,White alone,Black,American Indian,Asian,Pacific Islander,Other Race,Hispanic,Percent Hispanic,Percent White,Percent Black,Percent American Indian,Percent Asian,Percent Pacific Islander,Percent Other Race,Tract
0,"POLYGON ((-71.81752 42.26785, -71.81746 42.268...",8079,7340,4906,887,31,887,2,627,1376,17.03,60.73,10.98,0.38,10.98,0.02,7.76,731600


In [24]:
# First drop the old subdivided tracts
worcester2020Census4326 = worcester2020Census4326.drop(tract_731601.index)
worcester2020Census4326 = worcester2020Census4326.drop(tract_731602.index)


In [25]:
Census4326 = pd.concat([worcester2020Census4326, combined_gdf], ignore_index=True)

In [26]:
Census4326.crs


<Geographic 2D CRS: EPSG:4326>
Name: WGS 84
Axis Info [ellipsoidal]:
- Lat[north]: Geodetic latitude (degree)
- Lon[east]: Geodetic longitude (degree)
Area of Use:
- name: World.
- bounds: (-180.0, -90.0, 180.0, 90.0)
Datum: World Geodetic System 1984 ensemble
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

In [27]:
Census4326['Tract'].nunique()

60

In [28]:
Census4326

,Total Pop,One race,White alone,Black,American Indian,Asian,Pacific Islander,Other Race,Hispanic,Percent Hispanic,Percent White,Percent Black,Percent American Indian,Percent Asian,Percent Pacific Islander,Percent Other Race,geometry,Tract
0,6690,6299,5675,189,9,298,1,127,290,4.30,83.80,2.70,0.00,4.40,0.00,0.70,"POLYGON ((-71.78310 42.21614, -71.78311 42.216...",737300
1,5374,4901,3425,836,10,323,0,307,620,11.50,61.30,15.00,0.10,6.00,0.00,1.50,"POLYGON ((-71.77344 42.24704, -71.77342 42.247...",732801
2,3436,3221,2850,90,3,179,1,98,204,5.90,81.90,2.60,0.00,5.20,0.00,0.70,"POLYGON ((-71.73124 42.24129, -71.73123 42.241...",761300
3,4835,4383,3123,703,16,296,0,245,529,10.90,62.50,14.30,0.30,6.10,0.00,1.20,"POLYGON ((-71.78041 42.25085, -71.77988 42.250...",732301
4,5381,5055,4744,119,5,130,4,53,227,4.20,86.90,2.00,0.10,2.40,0.10,0.30,"POLYGON ((-71.79705 42.20999, -71.79711 42.210...",737100
5,3113,2930,2656,92,5,125,0,52,150,4.80,83.60,3.00,0.10,4.00,0.00,0.40,"MULTIPOLYGON (((-71.81964 42.23033, -71.81777 ...",736400
6,7444,6510,3944,1420,32,349,0,765,1543,20.70,49.40,18.40,0.10,4.60,0.00,1.90,"POLYGON ((-71.82528 42.22993, -71.82510 42.230...",732901
7,4620,4139,2425,1003,8,353,0,350,783,16.90,49.00,20.80,0.00,7.60,0.00,1.60,"POLYGON ((-71.79223 42.23691, -71.79220 42.237...",732802
8,3299,3193,2470,227,0,345,3,148,396,12.00,68.90,6.30,0.00,10.50,0.10,0.00,"POLYGON ((-71.81283 42.24045, -71.81276 42.240...",732902
9,4699,4158,1921,1206,27,191,4,809,1460,31.10,34.10,23.80,0.20,4.10,0.00,2.00,"POLYGON ((-71.80179 42.24085, -71.80157 42.240...",732700


In [29]:
import folium

# Create a Folium map
m = folium.Map(location=[42.2626, -71.8023], zoom_start=10)
colormap = LinearColormap(colors=['green', 'white'], vmin=17.7, vmax=88.7)

# Iterate over the rows of the worcesterCensus4326 GeoDataFrame and add polygons to the map with color based on the "Percent White" column
for idx, row in Census4326.iterrows():
    # Check the value of "Percent White" for the current row
    print(f"Census Tract: {row['Tract']}, Percent White: {row['Percent White']}")
    
    # Style function for GeoJSON features
    style_function = lambda x, row=row: {
        'fillColor': colormap(row['Percent White']),
        'color': 'black',
        'weight': 1,
        'fillOpacity': 0.7
    }
    # Add GeoJSON feature to the map
    #folium.GeoJson(row.geometry.__geo_interface__, style_function=style_function).add_to(m)
    # Add GeoJSON feature to the map with tooltip showing "Percent White" value
    
    folium.GeoJson(row.geometry.__geo_interface__, 
                   style_function=style_function,
                   tooltip=f"Census Tract: {row['Tract']}<br>Percent White: {row['Percent White']}").add_to(m)
    
#folium.GeoJson(combined_gdf.geometry).add_to(m)
# Display the map
m


Census Tract: 737300, Percent White: 83.8
Census Tract: 732801, Percent White: 61.3
Census Tract: 761300, Percent White: 81.9
Census Tract: 732301, Percent White: 62.5
Census Tract: 737100, Percent White: 86.9
Census Tract: 736400, Percent White: 83.6
Census Tract: 732901, Percent White: 49.4
Census Tract: 732802, Percent White: 49.0
Census Tract: 732902, Percent White: 68.9
Census Tract: 732700, Percent White: 34.1
Census Tract: 733000, Percent White: 26.3
Census Tract: 732600, Percent White: 31.2
Census Tract: 731300, Percent White: 17.7
Census Tract: 732302, Percent White: 47.6
Census Tract: 739101, Percent White: 64.1
Census Tract: 732202, Percent White: 60.0
Census Tract: 732201, Percent White: 55.6
Census Tract: 732002, Percent White: 53.7
Census Tract: 731900, Percent White: 28.8
Census Tract: 730402, Percent White: 38.6
Census Tract: 730500, Percent White: 50.0
Census Tract: 730401, Percent White: 40.3
Census Tract: 730300, Percent White: 64.6
Census Tract: 730200, Percent Whit

In [30]:
# Check the range of values in the "Percent White" column
print(Census4326['Percent White'].describe())

count    60.000000
mean     57.295500
std      21.194576
min      17.700000
25%      41.050000
50%      60.365000
75%      76.050000
max      88.700000
Name: Percent White, dtype: float64


In [31]:
#Census4326.to_csv("wooCensus2020.csv")

In [32]:
# For this project we are using 32619, so transform the CRS before downloading as .shp file
wooCensus2020 = Census4326.to_crs(epsg=32619)

In [33]:
wooCensus2020.crs

<Projected CRS: EPSG:32619>
Name: WGS 84 / UTM zone 19N
Axis Info [cartesian]:
- E[east]: Easting (metre)
- N[north]: Northing (metre)
Area of Use:
- name: Between 72°W and 66°W, northern hemisphere between equator and 84°N, onshore and offshore. Aruba. Bahamas. Brazil. Canada - New Brunswick (NB); Labrador; Nunavut; Nova Scotia (NS); Quebec. Colombia. Dominican Republic. Greenland. Netherlands Antilles. Puerto Rico. Turks and Caicos Islands. United States. Venezuela.
- bounds: (-72.0, 0.0, -66.0, 84.0)
Coordinate Operation:
- name: UTM zone 19N
- method: Transverse Mercator
Datum: World Geodetic System 1984 ensemble
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

In [34]:
wooCensus2020.to_file("wooCensus2020.shp")